#### **Var definitions**
Change it whenever necessary

In [5]:
access_oauth_token = "token <PLACE_HOLDER>" #TODO: delete the value before commiting the code
repo = "ipno-llead/processing"
out = '/Users/lorenapacheco/Concordia/se4ai_class/se4a1_class_group_project/bug-reports/'
repo_name= repo.split('/')[1]
proj = repo_name
base_path = '/Users/lorenapacheco/Concordia/se4ai_class/'
out_commits_dir = base_path + "se4a1_class_group_project/commits/" + repo_name
proj_path = base_path + "systems_being_studied/" + repo_name

#### **Bug reports mining**
Getting all the Github **issues** from the project that are **closed** and have the **bug**'s label

In [6]:
import os
import json
import requests
import pprint

def get_git_closed_bug_issues(repo, media_type = "application/vnd.github+json"):
    issue_numbers_list = []
    current_page=1
    pagination_end = False
    
    #iteration throught the pagination
    while not pagination_end:
        issues_json = requests.get(
            "https://api.github.com/repos/"+repo+"/issues?state=closed&labels=bug&page="+str(current_page),
            headers={
                'Accept': media_type,
                'Authorization': access_oauth_token
            }
        ).json()
        if len(issues_json) > 0:
            for issue in issues_json:
                issue_numbers_list.append(issue["number"])
            current_page = current_page + 1
        else:
            pagination_end = True
            
    return issue_numbers_list


issue_numbers_list = get_git_closed_bug_issues(repo)
print(issue_numbers_list)
print("\nNumber of issues: " + str(len(issue_numbers_list)))

[390, 389, 388, 387, 383, 382, 380, 379, 378, 377, 372, 371, 370, 369, 368, 367, 366, 364, 362, 361, 359, 357, 353, 351, 349, 348, 347, 344, 343, 342, 341, 327, 326, 325, 324, 314, 313, 309, 306, 304, 300, 299, 297, 293, 291, 288, 286, 285, 284, 278, 272, 270, 266, 260, 254, 251, 243, 235, 224, 218, 214, 206, 200, 184, 182, 181, 177, 173, 171, 170, 156, 155, 142, 137, 130, 126, 121, 115, 105, 104, 98, 92, 82, 78, 77, 69, 64, 63, 61, 50, 49, 45, 44, 43, 42, 41, 40, 39, 36, 35, 33, 32, 31, 22, 15, 14, 10]

Number of issues: 107


#### **Bug reports content extraction**
Extraction the content of each of these bug reports

In [7]:
output_folder = out + repo_name+ '/'

def git_request_br_json(issue_n, repo, media_type = 'application/vnd.github.v3+json'):
    retrieved_json = requests.get(
        'https://api.github.com/repos/'+repo+'/issues/'+str(issue_n),
        headers={
            'Accept': media_type,
            'Authorization': access_oauth_token
        }
    ).json()
    #pprint.pprint(retrieved_json)
    
    if 'comments_url' not in retrieved_json:
        return
    
    comment_url = retrieved_json['comments_url']
    retrieved_comments = requests.get(
        comment_url,
        headers={
            'Accept': media_type,
            'Authorization': access_oauth_token
        }
    ).json()
    retrieved_json['comments_content'] = retrieved_comments
    return retrieved_json

def dict_to_json_file(file, dic, folder=output_folder):
    if not os.path.exists(folder):
        os.makedirs(folder)
    with open(os.path.join(folder, file+'.json'), 'w') as fp:
        json.dump(dic, fp, sort_keys=True, indent=4)
    fp.close()


for issue_number in issue_numbers_list:
    try:
        retrieved_json = git_request_br_json(issue_number, repo)
    except ConnectionError:
        print('ConnectionError', repo, issue_number)
        break

    if not retrieved_json:
        print('Json not retrieved, something might be wrong')
        continue

    dict_to_json_file(repo_name+'-'+str(issue_number), retrieved_json)
print ("Extraction completed")

Extraction completed


#### **Commits search**
Searching for the respective commits

In [8]:
import re
import glob
from collections import defaultdict
from urllib.parse import quote

bug_reports_commits = defaultdict(dict)
cont = 0

def get_bug_report_commit(bug, proj_path):
    os.chdir(proj_path)
    log_grep_command = "git log --grep=\"#"+ str(bug)+"[^0-9]\" --grep=\"#"+ str(bug)+"$\" --pretty=format:\"%H\" >> commit_output"
    #log_grep_command = "git log --grep="+ str(bug)+" --pretty=format:\"%H\" >> commit_output"
    os.system(log_grep_command)
    lines = []
    with open("commit_output", 'r') as fp:
        lines =fp.readlines()
        fp.close()
    os.system("rm commit_output")
    if (lines):
        commit_hash = lines[0]
        return(commit_hash)
    return ""

for bug in issue_numbers_list:
    print(bug)
    commit = get_bug_report_commit(bug, proj_path)
    if (commit != ""):
        content = {
            "commit": commit.strip()
        }
        bug_reports_commits[bug] = content
        cont = cont + 1


if bug_reports_commits:
    dict_to_json_file("bug_reports_with_commits", bug_reports_commits, out_commits_dir)

print("Extraction complete")
print("Number obtained: " + str(cont))

390
389
388
387
383
382
380
379
378
377
372
371
370
369
368
367
366
364
362
361
359
357
353
351
349
348
347
344
343
342
341
327
326
325
324
314
313
309
306
304
300
299
297
293
291
288
286
285
284
278
272
270
266
260
254
251
243
235
224
218
214
206
200
184
182
181
177
173
171
170
156
155
142
137
130
126
121
115
105
104
98
92
82
78
77
69
64
63
61
50
49
45
44
43
42
41
40
39
36
35
33
32
31
22
15
14
10
Extraction complete
Number obtained: 94
